In [1]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

In [2]:
def preprocessing(filename):
    path = '/content/'+filename+'.txt'
    data = pd.read_csv(path, sep = ';')
    hos = []
    for i in range(len(data.emotion)):
        if data['emotion'][i] in ['surprise','joy', 'love']:
            hos.append(1) # 1 is not suffering from mental health
        else:
            hos.append(0) # 0 is suffering from mental health
    data['hos'] = hos
    return data

In [3]:
train_data = preprocessing('train')
train = train_data.copy()
train['emotion'].value_counts()

joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: emotion, dtype: int64

In [4]:
train.groupby('hos').describe()

text         ...  emotion      
    count unique  ...      top  freq
hos               ...               
0    8762   8754  ...  sadness  4666
1    7238   7221  ...      joy  5362

[2 rows x 8 columns]

In [5]:
model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(model, output_shape=[20], input_shape=[], 
                           dtype=tf.string, trainable=True)

In [6]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])

In [9]:
val= preprocessing('val')
val.head()

,text,emotion,hos
0,im feeling rather rotten so im not very ambiti...,sadness,0
1,im updating my blog because i feel shitty,sadness,0
2,i never make her separate from me because i do...,sadness,0
3,i left with my bouquet of red and yellow tulip...,joy,1
4,i was feeling a little vain when i did this one,sadness,0


In [10]:
history = model.fit(train.text,
                    train.hos,
                    epochs=40,
                    batch_size=512,
                    validation_data=(val.text, val.hos),
                    verbose = 0)

In [11]:
test = preprocessing('val')
results = model.evaluate(test['text'], test['hos'])

print(results)

63/63 [==============================] - 0s 3ms/step - loss: 0.3446 - accuracy: 0.8630
[0.3445972502231598, 0.8629999756813049]


In [12]:
predstr = model.predict(train.text)

In [13]:
print(predstr.max())
print(predstr.min())

43.391766
-17.053513


In [15]:
model.save('/content/mentalhealthmodel')
!zip -r /content/mentalhealthmodel.zip /content/mentalhealthmodel

INFO:tensorflow:Assets written to: /content/mentalhealthmodel/assets


INFO:tensorflow:Assets written to: /content/mentalhealthmodel/assets


  adding: content/mentalhealthmodel/ (stored 0%)
  adding: content/mentalhealthmodel/saved_model.pb (deflated 86%)
  adding: content/mentalhealthmodel/assets/ (stored 0%)
  adding: content/mentalhealthmodel/assets/tokens.txt (deflated 63%)
  adding: content/mentalhealthmodel/keras_metadata.pb (deflated 84%)
  adding: content/mentalhealthmodel/variables/ (stored 0%)
  adding: content/mentalhealthmodel/variables/variables.data-00000-of-00001 (deflated 44%)
  adding: content/mentalhealthmodel/variables/variables.index (deflated 60%)


In [16]:
import keras
mentalhealthmodel = keras.models.load_model('/content/mentalhealthmodel')

In [17]:
def postprocessor(preds):
  range = 43.391766-(-17.053513)
  norm_preds = []
  probab = []
  for i in preds:
    norm_preds.append((i - (-17.053513)) / range)
    probab.append((i - (-17.053513)) * 100 / range)
  return np.mean(probab)

In [18]:
answers = []
answers.append(input('please say anything :'))
results = mentalhealthmodel.predict(answers)
print('Your mental health score is:', postprocessor(results))

please say anything :lonely alone bad cry depressed lonely alone bad cry depressed lonely alone bad cry depressed
Your mental health score is: -9.446291
